In [ ]:
import sys
sys.path.append('../bggcomplex')

from fast_module import BGGCohomology
from bggcomplex import BGGComplex
from quantum_center_module_v2 import *
from tqdm.auto import tqdm

# the parameters we actually want to change
BGG = BGGComplex('B3')
subset=[]
s = 0

# compute only half of the table, extend by symmetry
half_only = True
extend_half = half_only

# Display in full form
compact = False

ab_dic = dict()
with tqdm(all_abijk(BGG,s=s,subset=subset,half_only=half_only)) as inner_pbar:
    with tqdm(leave=None) as outer_pbar:
        map_pbar = tqdm()
        for a,b,i,j,k in inner_pbar:
            if a!=b:
                inner_pbar.set_description('i+j= %d, j-i = %d'%(a,b))
                coker = Tijk_basis(BGG, i,j,k,subset=subset,pbar=outer_pbar,kernel_benchmark=True,ker_method='fast')
                outer_pbar.set_description('Initializing cohomology')
                cohom = BGGCohomology(BGG, Mijk(BGG,i,j,k,subset=subset), 
                                      coker=coker,pbars = [outer_pbar,map_pbar])
                outer_pbar.set_description('Computing cohomology')
                cohom_string = cohom.cohomology_LaTeX(i=i,only_strings=True,compact=compact)
                ab_dic[(a,b)]=cohom_string
display_ab_dic(ab_dic,extend_half = extend_half)

In [ ]:
tijk1 = Tijk_basis(BGG, 4,6,-10,subset=subset,pbar=outer_pbar,kernel_benchmark=True,ker_method='slow')
tijk2 = Tijk_basis(BGG, 4,6,-10,subset=subset,pbar=outer_pbar,kernel_benchmark=True,ker_method='fast')
for k in tijk1.keys():
    if tijk1[k].nrows()==0:
        print(k,tijk1[k].nrows()-tijk2[k].nrows(),tijk1[k].ncols()-tijk2[k].ncols())

In [ ]:
tijk2[-12,-6].nrows()

In [ ]:
import csv
import os 
import numpy as np
mat_path = os.path.join('mat_benchmark','mat','(-6, -3)_10.csv')
mat = np.genfromtxt(mat_path, delimiter=',')
mat = Matrix(mat,ring=ZZ)
print(mat.nrows(),mat.ncols(),len(mat.nonzero_positions()))

%time mat.right_kernel().basis_matrix()

In [ ]:
ker = mat.__pari__().matker(flag=1).mattranspose().sage()